In [1]:
backup_dir = '/root/datos/maestria/netopaas/luca_explore/surgeries'
name = 'Deng'
cell_key = 'cell_type_adjusted'
integ_features = 3000

Based on SCTransform [v2 vignette](https://satijalab.org/seurat/archive/v4.3/sctransform_v2_vignette)

In [2]:
import anndata as ad
import pandas as pd
import anndata2ri
anndata2ri.activate()
%load_ext rpy2.ipython

/tmp/ipykernel_23522/3724497324.py:4: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()


In [3]:
adata  = ad.read_h5ad(f'{backup_dir}/{name}/query.h5ad')

In [4]:
preds = pd.read_csv(f'{backup_dir}/{name}_predicted_leiden.csv')
preds.index = adata.obs.index
adata.obs[cell_key] = preds[cell_key]
adata.obs

,orig.ident,nCount_RNA,nFeature_RNA,percent.mt,assigned_cell_type,stage,Ecotype,ecoRecovery,Patient Number,Group,...,T stage,N stage,M stage,Pathological stage,cell_type_tumor,dataset,sample,_scvi_batch,_scvi_labels,cell_type_adjusted
index,,,,,,,,,,,,,,,,,,,,,
0,P001,4348.0,2099,12.531686,5,1,unassigned,unassigned,P001,pGGO,...,T1a,N0,M0,IA1,Unknown,P001,P001,12,51,Mast cell
1,P001,5701.0,2614,4.821479,0,1,unassigned,CD8.T.cells_S04,P001,pGGO,...,T1a,N0,M0,IA1,Unknown,P001,P001,12,51,T cell CD8 effector memory
2,P001,5636.0,2489,2.784740,0,1,unassigned,CD4.T.cells_S04,P001,pGGO,...,T1a,N0,M0,IA1,Unknown,P001,P001,12,51,T cell CD4
3,P001,5991.0,2591,6.061365,0,1,unassigned,CD4.T.cells_S05,P001,pGGO,...,T1a,N0,M0,IA1,Unknown,P001,P001,12,51,T cell CD8 activated
4,P001,3464.0,1625,9.162717,5,1,unassigned,Mast.cells_S01,P001,pGGO,...,T1a,N0,M0,IA1,Unknown,P001,P001,12,51,Mast cell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421706,P049,1497.0,723,0.407106,1,9,unassigned,unassigned,P049,SolidN,...,T1b,N2,M0,IIIA,Unknown,P049,P049,60,51,Macrophage
421707,P049,7093.0,541,0.027082,4,9,unassigned,unassigned,P049,SolidN,...,T1b,N2,M0,IIIA,Unknown,P049,P049,60,51,Plasma cell
421708,P049,5584.0,2466,5.115492,4,9,unassigned,unassigned,P049,SolidN,...,T1b,N2,M0,IIIA,Unknown,P049,P049,60,51,B cell


In [5]:
%Rdevice svg

In [6]:
%%R -i adata -i backup_dir -i name
library(Seurat)
adata.seurat <- as.Seurat(adata, data = "X", counts='X')
SaveSeuratRds(adata.seurat, file.path(backup_dir,name,"query.rds"))


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

Loading required package: SeuratObject
Loading required package: sp

Attaching package: ‘sp’

The following object is masked from ‘package:IRanges’:

    %over%


Attaching package: ‘SeuratObject’

The following object is masked from ‘package:SummarizedExperiment’:

    Assays

The following object is masked from ‘package:GenomicRanges’:

    intersect

The following object is masked from ‘package:GenomeInfoDb’:

    intersect

The following object is masked from ‘package:IRanges’:

    intersect

The following object is masked from ‘package:S4Vectors’:

    intersect

The following object is masked from ‘package:BiocGenerics’:

    intersect

The following objects are masked from ‘package:base’:

    intersect, t


Attaching package: ‘Seurat’

The following object is masked from ‘package:SummarizedExperiment’:

    Assays



SCTransform processing: 2008.358 sec elapsed

In [ ]:
%%R -i integ_features
library(Seurat)
library(dplyr)
library(purrr)
library(tictoc)

# Start the timer
tic("Total execution time")

# Split and transform the Seurat objects
# tic("SCTransform processing")
# adata.seurat.list <- adata.seurat %>%
#   SplitObject(split.by = "sample") %>%
#   map(~ SCTransform(.x, vst.flavor = "v2", verbose = FALSE, assay='originalexp', return.only.var.genes = FALSE)) # Important to return all genes or else integration does not work
# toc()  # Ends SCTransform timing

# Proceed with further analysis (e.g., integration)
tic("Integration processing")
features <- SelectIntegrationFeatures(object.list = adata.seurat.list, nfeatures = integ_features)

adata.seurat.list <- PrepSCTIntegration(object.list = adata.seurat.list, anchor.features = features)
anchors <- FindIntegrationAnchors(
  object.list = adata.seurat.list,
  normalization.method = "SCT",
  anchor.features = features
)
integrated_data <- IntegrateData(
  anchorset = anchors,
  normalization.method = "SCT"
)
toc()  # Ends Integration timing

toc()  # Ends total execution timing
integrated_data

  |                                                  | 0 % ~calculating   |++                                                | 2 % ~19s           |+++                                               | 4 % ~26s           |++++                                              | 6 % ~24s           |+++++                                             | 8 % ~23s           |++++++                                            | 10% ~20s           |+++++++                                           | 12% ~18s           |++++++++                                          | 14% ~18s           |+++++++++                                         | 16% ~18s           |++++++++++                                        | 18% ~18s           |+++++++++++                                       | 20% ~18s           |++++++++++++                                      | 22% ~17s           |+++++++++++++                                     | 24% ~17s           |++++++++++++++                                    | 27% ~17s 

### Import new annots

In [ ]:
2+2

In [12]:
preds = pd.read_csv(f'{backup_dir}/{name}_predicted_leiden.csv')

In [16]:
%%R -i preds
integrated_data@meta.data$cell_type_adjusted <- preds$cell_type_adjusted

## Markers

In [ ]:
%%R
tic()
integrated_data <- PrepSCTFindMarkers(integrated_data)
integrated_data@meta.data$cell_type_adjusted <- as.factor(integrated_data@meta.data$cell_type_adjusted)
Idents(object=integrated_data) <- 'cell_type_adjusted'
markers <- FindMarkers(object = integrated_data, ident.1 = "Tumor LUAD_mixed", assay='SCT', recorrect_umi=F) # We dont recollect UMI because we use UMI counts the SCRAssay method
print(head(x = markers, n=15))
toc()